# Noisepowertemp1 - Demo

Noisepowertemp1.py provides the total noise temperature and total noise power of circuit and its components. This is obtained by providing the voltage fluctuations data of resistor and each amplifier, gains from each amplifier and the frequency, bandwidth and resistance of the circuit.

### Inputs:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.constants import *

In [2]:
class Amplifier:
    def __init__(self, gain, vflucation_data_path):
        self.gain = gain
        self.vflucation_data_path = vflucation_data_path

class Resistor:
    def __init__(self, vflucation_data_path):
        self.vflucation_data_path = vflucation_data_path

class Otherinputs:
    def __init__(self, resistance, bandwidth, frequency):
        self.resistance = resistance
        self.bandwidth = bandwidth
        self.frequency = frequency

In [3]:
#1st coloumn of the amplifiers class contain gains for each amplifier respectively and in the second coloumn we assign voltage fluctuations data of each amplifiers.  
amplifiers = [
    Amplifier(5,'Amplifier0_Voltageflucations.txt'),
    Amplifier(10,'Amplifier1_Voltageflucations.txt'),
    Amplifier(15,'Amplifier2_Voltageflucations.txt'),
    Amplifier(20,'Amplifier3_Voltageflucations.txt')
]

#Assigning voltage fluctuations data of resistor
resistor = [
    Resistor('Resistor_Voltageflucations.txt')
]

#otherinputs contain: resistance, frequency, bandwidth
otherinputs = [
    Otherinputs(50, 5e9, 1e9),
]

In [4]:
#providing the inputs
def inputs():
    #At the moment the model is designed for only 1 resistors, not for intermidate resistors
    
    vrms_of_resistor = []
    a = 1                             #random variables only used for constructing the loop
    b = []
    for i in range(len(resistor)):  
        b.append(np.loadtxt(resistor[i].vflucation_data_path))
        a = np.std(b)
        vrms_of_resistor.append(a)
        
        
        
    vrms_of_amplifiers = []
    gain_of_amplifiers = []
    a = 1                             #random variables only used for constructing the loop
    b = []
    for i in range(len(amplifiers)):
        b.append(np.loadtxt(amplifiers[i].vflucation_data_path))
        a = np.std(b)
        vrms_of_amplifiers.append(a)
        gain_of_amplifiers.append(amplifiers[i].gain)
    
    resistance = otherinputs[0].resistance
    bandwidth = otherinputs[0].bandwidth
    frequency = otherinputs[0].frequency
    
    return(vrms_of_resistor, vrms_of_amplifiers, gain_of_amplifiers, resistance, bandwidth, frequency)

vrms_of_resistor, vrms_of_amplifiers, gain_of_amplifiers, resistance, bandwidth, frequency = inputs()


#for calculating noiseopower of resistors
def noisepower_resistors(vrms_of_resistor, resistance):   
    

    #calculating noiseopower of resistors without gain
    noisepower_resistors_nogain = []
    for i in range(len(vrms_of_resistor)):
        noisepower_resistors_nogain.append(((vrms_of_resistor[i])**2)/(4*resistance))


    #calculating noiseopower of resistors with gain
    ll = 1                                             #random variables only used for constructing the loop
    rr = 1
    oo = 1
    mm = 1
    noisepower_resistor = []
    for i in range(len(vrms_of_resistor)):
        oo = noisepower_resistors_nogain[i]
        for j in range(0,len(gain_of_amplifiers)):  #equation for n amplifers, e.g. n=4; total noise power contributing from the resistor =  g1*g2*g3*g4(Noisepower of resistor)
            mm = gain_of_amplifiers[j]*oo
            oo = mm
        noisepower_resistor.append(oo)

    totalnoisepower_from_resisitors = sum(noisepower_resistor)   #total noisepower from the resistors
    
    return(noisepower_resistors_nogain, noisepower_resistor, totalnoisepower_from_resisitors)

noisepower_resistors_nogain, noisepower_resistor, totalnoisepower_from_resisitors = noisepower_resistors(vrms_of_resistor, resistance)




#for calculating noiseopower of amplifiers
def noisepower_amplifiers(vrms_of_amplifiers,gain_of_amplifiers, resistance):
    
    #calculating noiseopower of amplifiers without gain
    noisepower_amplifiers_nogain = []        
    for i in range(len(vrms_of_amplifiers)):
        noisepower_amplifiers_nogain.append(((vrms_of_amplifiers[i])**2)/(4*resistance))

    
    #calculating noiseopower of amplifiers with gain
    noisepower_amplifiers = []
    o = 1                                               #random variables only used for constructing the loop
    m = 1
    for i in range(len(vrms_of_amplifiers)):
        o = (((vrms_of_amplifiers[i])**2)/(4*resistance))
        for j in range(i+1,len(gain_of_amplifiers)):         #equation for n amplifers, e.g. n=4; total noise power contributing from the amplifer1 =  g2*g3*g4(Noisepower Amp1), total noise power contributing from the amplifer2 =  g3*g4(Noisepower Amp2), so on ....
            m = gain_of_amplifiers[j]*o
            o = m
        noisepower_amplifiers.append(o)
    
    totalnoisepower_from_amplifiers = sum(noisepower_amplifiers)   #total noisepower from the amplifiers

    
    return(noisepower_amplifiers_nogain,noisepower_amplifiers, totalnoisepower_from_amplifiers)

noisepower_amplifiers_nogain, noisepower_amplifiers, totalnoisepower_from_amplifiers = noisepower_amplifiers(vrms_of_amplifiers,gain_of_amplifiers, resistance)





#for calculating noisetemperatures of amplifiers
def noisetemperature_amplifiers(vrms_of_amplifiers, noisepower_amplifiers_nogain, noisepower_amplifiers, frequency, bandwidth):
    
    #temperature_amplifiers: are the actual temperatures(i.e by which we deduce the Vrms of each amplifer). This is just required for verifying the model. 
    t1 = 1
    temperature_amplifiers=[]
    for i in range(len(vrms_of_amplifiers)):
        t1 = (h*frequency)/(k*np.log(((h*frequency*bandwidth)/noisepower_amplifiers_nogain[i])+1))    #Formula for finding the temperature with given noisepower
        temperature_amplifiers.append(t1)
    
    #for calculating the noise temperature of the amplifiers
    t2 = 1
    noisetemperature_amplifiers=[]
    for i in range(len(vrms_of_amplifiers)):
        t2 = (h*frequency)/(k*np.log(((h*frequency*bandwidth)/noisepower_amplifiers[i])+1))
        noisetemperature_amplifiers.append(t2)
        
    
    totalnoisetemperature_from_amplifiers = sum(noisetemperature_amplifiers)

    
    return(temperature_amplifiers, noisetemperature_amplifiers, totalnoisetemperature_from_amplifiers)


temperature_amplifiers, noisetemperature_amplifiers, totalnoisetemperature_from_amplifiers = noisetemperature_amplifiers(vrms_of_amplifiers, noisepower_amplifiers_nogain, noisepower_amplifiers, frequency, bandwidth)





#for calculating the noise temperature of the resistor
def noisetemperature_resistors(vrms_of_resistor, frequency, bandwidth, noisepower_resistor):
    

    t3 = 1
    noisetemperature_resistors=[]
    for i in range(len(vrms_of_resistor)):
        t3 = (h*frequency)/(k*np.log(((h*frequency*bandwidth)/noisepower_resistor[i])+1))
        noisetemperature_resistors.append(t3)

    
    totalnoisetemperature_from_resisitors = sum(noisetemperature_resistors)
    return(noisetemperature_resistors, totalnoisetemperature_from_resisitors)


noisetemperature_resistors, totalnoisetemperature_from_resisitors = noisetemperature_resistors(vrms_of_resistor, frequency, bandwidth, noisepower_resistor)





def total_noise_temp_power(totalnoisepower_from_resisitors, totalnoisepower_from_amplifiers, totalnoisetemperature_from_resisitors, totalnoisetemperature_from_amplifiers):
    
    total_noisepower = totalnoisepower_from_resisitors  + totalnoisepower_from_amplifiers 


    total_noisetemperature = totalnoisetemperature_from_resisitors + totalnoisetemperature_from_amplifiers
    
    return(total_noisepower, total_noisetemperature)

total_noisepower, total_noisetemperature = total_noise_temp_power(totalnoisepower_from_resisitors, totalnoisepower_from_amplifiers, totalnoisetemperature_from_resisitors, totalnoisetemperature_from_amplifiers)

In [5]:
print('Noise power of the resistors:', noisepower_resistor)
print('---------------------------------------------------------------')
print('Total noise power of the resistors:', totalnoisepower_from_resisitors)
print('---------------------------------------------------------------')
    
print('Noise power of the amplifiers:', noisepower_amplifiers)
print('---------------------------------------------------------------')
    
    
print('Total noise power of the amplifiers:', totalnoisepower_from_amplifiers)
print('---------------------------------------------------------------')
    
    
print('Total Noisetemperature from the amplifiers:', totalnoisetemperature_from_amplifiers)
print('---------------------------------------------------------------')
    
    
print('Total Noisetemperature from the resistors:', noisetemperature_resistors)
print('---------------------------------------------------------------')
    
    
print('Total Noisepower of circuit:', total_noisepower)
print('Total Noisetemperature of circuit:', total_noisetemperature)
    

Noise power of the resistors: [4.1293317920038766e-08]
---------------------------------------------------------------
Total noise power of the resistors: 4.1293317920038766e-08
---------------------------------------------------------------
Noise power of the amplifiers: [4.121457901358969e-09, 3.616016480423405e-10, 2.0665339458204495e-11, 8.607290446105849e-13]
---------------------------------------------------------------
Total noise power of the amplifiers: 4.504585617904124e-09
---------------------------------------------------------------
Total Noisetemperature from the amplifiers: 65253.257607772706
---------------------------------------------------------------
Total Noisetemperature from the resistors: [598172.5926191799]
---------------------------------------------------------------
Total Noisepower of circuit: 4.579790353794289e-08
Total Noisetemperature of circuit: 663425.8502269526


# Noisepowertemp2 - Demo

Noisepowertemp2.py provides the total noise temperature and total noise power of circuit and its components. This is obtained by providing the voltage(rms) of resistor and each amplifiers, with gains from each amplifier and the frequency, bandwidth and resistance of the circuit. 

The difference between the Noisepowertemp1 and Noisepowertemp2 is that, here we need to provide the voltage-rms instead of voltage fluctuation data points. 

### Inputs:

In [6]:
class Amplifier:
    
    def __init__(self, gain, vrms):
        self.gain = gain
        self.vrms = vrms


class Resistor:
    
    def __init__(self, vrms):
        self.vrms = vrms
        
class Circuit:
    def __init__(self, resistance, bandwidth, frequency):
        self.resistance = resistance
        self.bandwidth = bandwidth
        self.frequency = frequency
        
        
#1st coloumn of the amplifiers class contain gains for each amplifier respectively and in the second coloumn we assign voltage(rms) of each amplifiers.  
amplifiers = [
    Amplifier(5,1.1745930046241017e-05),
    Amplifier(10,1.017186676745961e-05),
    Amplifier(15,9.097630153982875e-06),
    Amplifier(20,8.304629751072748e-06)
]
 
    
#Assigning voltage(rms) of resistor
resistor = [
    Resistor(2.3493270156603197e-05)
]    

#otherinputs contain: resistance, frequency, bandwidth
circuit_inputs = [
    Circuit(50, 5e9, 1e9),
]

In [7]:
#providing the inputs
def inputs():
    #At the moment the model is designed for only 1 resistors, not for intermidate resistors
    
    
    
    vrms_of_resistor = []
                              
    b = []                    #random variables only used for constructing the loop
    for i in range(len(resistor)):  
        b.append(resistor[i].vrms)
        vrms_of_resistor.append(b)
        
        
        
    vrms_of_amplifiers = []
    gain_of_amplifiers = []
    
                
    b = []       #random variables only used for constructing the loop
    for i in range(len(amplifiers)):
        b.append(amplifiers[i].vrms)
        vrms_of_amplifiers.append(b)
        gain_of_amplifiers.append(amplifiers[i].gain)
    
    resistance = circuit_inputs[0].resistance
    bandwidth = circuit_inputs[0].bandwidth
    frequency = circuit_inputs[0].frequency
    
    return(vrms_of_resistor, vrms_of_amplifiers, gain_of_amplifiers, resistance, bandwidth, frequency)




vrms_of_resistor, vrms_of_amplifiers, gain_of_amplifiers, resistance, bandwidth, frequency = inputs()




#for calculating noiseopower of resistors
def noisepower_resistors(vrms_of_resistor, gain_of_amplifiers, resistance):   
    

    #calculating noiseopower of resistors without gain
    noisepower_resistors_nogain = []
    for i in range(len(vrms_of_resistor)):
        noisepower_resistors_nogain.append(((resistor[i].vrms)**2)/(4*resistance))


    #calculating noiseopower of resistors with gain
    ll = 1                                             #random variables only used for constructing the loop
    rr = 1
    oo = 1
    mm = 1
    noisepower_resistor = []
    for i in range(len(vrms_of_resistor)):
        oo = noisepower_resistors_nogain[i]
        for j in range(0,len(gain_of_amplifiers)):  #equation for n amplifers, e.g. n=4; total noise power contributing from the resistor =  g1*g2*g3*g4(Noisepower of resistor)
            mm = gain_of_amplifiers[j]*oo
            oo = mm
        noisepower_resistor.append(oo)
    

    
    totalnoisepower_from_resisitors = sum(noisepower_resistor)   #total noisepower from the resistors

    
    return(noisepower_resistors_nogain, noisepower_resistor, totalnoisepower_from_resisitors)



noisepower_resistors_nogain, noisepower_resistor, totalnoisepower_from_resisitors = noisepower_resistors(vrms_of_resistor, gain_of_amplifiers, resistance)





#for calculating noiseopower of amplifiers
def noisepower_amplifiers(vrms_of_amplifiers, gain_of_amplifiers, resistance):
    
    #calculating noiseopower of amplifiers without gain
    noisepower_amplifiers_nogain = []        
    for i in range(len(vrms_of_amplifiers)):
        noisepower_amplifiers_nogain.append(((amplifiers[i].vrms)**2)/(4*resistance))

    
    #calculating noiseopower of amplifiers with gain
    noisepower_amplifiers = []
    o = 1                                               #random variables only used for constructing the loop
    m = 1
    for i in range(len(vrms_of_amplifiers)):
        o = (((amplifiers[i].vrms)**2)/(4*resistance))
        for j in range(i+1,len(gain_of_amplifiers)):         #equation for n amplifers, e.g. n=4; total noise power contributing from the amplifer1 =  g2*g3*g4(Noisepower Amp1), total noise power contributing from the amplifer2 =  g3*g4(Noisepower Amp2), so on ....
            m = gain_of_amplifiers[j]*o
            o = m
        noisepower_amplifiers.append(o)
        

    
    totalnoisepower_from_amplifiers = sum(noisepower_amplifiers)   #total noisepower from the amplifiers

    
    return(noisepower_amplifiers_nogain,noisepower_amplifiers, totalnoisepower_from_amplifiers)



noisepower_amplifiers_nogain, noisepower_amplifiers, totalnoisepower_from_amplifiers = noisepower_amplifiers(vrms_of_amplifiers, gain_of_amplifiers, resistance)



#for calculating noisetemperatures of amplifiers
def noisetemperature_amplifiers(noisepower_amplifiers_nogain, noisepower_amplifiers, totalnoisepower_from_amplifiers, frequency, bandwidth, resistance, vrms_of_amplifiers):
    
    #temperature_amplifiers: are the actual temperatures(i.e by which we deduce the Vrms of each amplifer). This is just required for verifying the model. 
    t1 = 1
    temperature_amplifiers=[]
    for i in range(len(vrms_of_amplifiers)):
        t1 = (h*frequency)/(k*np.log(((h*frequency*bandwidth)/noisepower_amplifiers_nogain[i])+1))    #Formula for finding the temperature with given noisepower
        temperature_amplifiers.append(t1)
    
    #for calculating the noise temperature of the amplifiers
    t2 = 1
    noisetemperature_amplifiers=[]
    for i in range(len(vrms_of_amplifiers)):
        t2 = (h*frequency)/(k*np.log(((h*frequency*bandwidth)/noisepower_amplifiers[i])+1))
        noisetemperature_amplifiers.append(t2)
        
    
    
    totalnoisetemperature_from_amplifiers = sum(noisetemperature_amplifiers)

    
    return(temperature_amplifiers, noisetemperature_amplifiers, totalnoisetemperature_from_amplifiers)



temperature_amplifiers, noisetemperature_amplifiers, totalnoisetemperature_from_amplifiers = noisetemperature_amplifiers(noisepower_amplifiers_nogain, noisepower_amplifiers, totalnoisepower_from_amplifiers, frequency, bandwidth, resistance, vrms_of_amplifiers)



#for calculating the noise temperature of the amplifiers

def noisetemperature_resistors(frequency, bandwidth, noisepower_resistor,vrms_of_resistor):
    
    t3 = 1
    noisetemperature_resistors=[]
    for i in range(len(vrms_of_resistor)):
        t3 = (h*frequency)/(k*np.log(((h*frequency*bandwidth)/noisepower_resistor[i])+1))
        noisetemperature_resistors.append(t3)

    
    totalnoisetemperature_from_resisitors = sum(noisetemperature_resistors)
 
    return(noisetemperature_resistors, totalnoisetemperature_from_resisitors)

noisetemperature_resistors, totalnoisetemperature_from_resisitors = noisetemperature_resistors(frequency, bandwidth, noisepower_resistor, vrms_of_resistor)


def total_noise_temp_power(totalnoisepower_from_resisitors, totalnoisepower_from_amplifiers, totalnoisetemperature_from_resisitors, totalnoisetemperature_from_amplifiers):
    
    total_noisepower = totalnoisepower_from_resisitors  + totalnoisepower_from_amplifiers 


    total_noisetemperature = totalnoisetemperature_from_resisitors + totalnoisetemperature_from_amplifiers
    
    return(total_noisepower, total_noisetemperature)

total_noisepower, total_noisetemperature = total_noise_temp_power(totalnoisepower_from_resisitors, totalnoisepower_from_amplifiers, totalnoisetemperature_from_resisitors, totalnoisetemperature_from_amplifiers)



In [8]:
print('Noise power of the resistors:', noisepower_resistor)
print('---------------------------------------------------------------')
print('Total noise power of the resistors:', totalnoisepower_from_resisitors)
print('---------------------------------------------------------------')


print('Noise power of the amplifiers:', noisepower_amplifiers)
print('---------------------------------------------------------------')
print('Total noise power of the amplifiers:', totalnoisepower_from_amplifiers)
print('---------------------------------------------------------------')


print('Noisetemperature of the amplifiers:', noisetemperature_amplifiers)
print('---------------------------------------------------------------')
print('Total Noisetemperature from the amplifiers:', totalnoisetemperature_from_amplifiers)
print('---------------------------------------------------------------')


print('Total Noisetemperature from the resistors:', noisetemperature_resistors)
print('---------------------------------------------------------------')


print('Total Noisepower of circuit:', total_noisepower)
print('Total Noisetemperature of circuit:', total_noisetemperature)

Noise power of the resistors: [4.139503069883569e-08]
---------------------------------------------------------------
Total noise power of the resistors: 4.139503069883569e-08
---------------------------------------------------------------
Noise power of the amplifiers: [2.0695030897678123e-09, 1.5520031030242383e-10, 8.276687441865845e-12, 3.448343765120131e-13]
---------------------------------------------------------------
Total noise power of the amplifiers: 2.233324921888614e-09
---------------------------------------------------------------
Noisetemperature of the amplifiers: [29978.723719530713, 2248.246539334763, 119.91960083864745, 5.019208418431708]
---------------------------------------------------------------
Total Noisetemperature from the amplifiers: 32351.909068122553
---------------------------------------------------------------
Total Noisetemperature from the resistors: [599645.9975998902]
---------------------------------------------------------------
Total Noisepow